In [ ]:
cd ..

In [2]:
from pathlib import Path
import pandas as pd

In [ ]:
def get_color(player, game):
    player = player.lower()
    if player in game["White"].split(",")[0].lower():
        return "White"
    elif player in game["Black"].split(",")[0].lower():
        return "Black"
    else:
        raise Exception("Player not in game")

In [ ]:
def get_cp_loss(player):
    cp = pd.read_csv(f"output/centipawns/{player}.csv", index_col=0)
    cp = cp.clip(-1000, 1000)
    diffs = cp.diff(axis=1)
    md = pd.read_csv(f"output/metadata/{player}.csv", index_col=0)
    colors = [get_color(player, row) for _, row in md.iterrows()]
    all_diffs = []
    for i in range(cp.shape[0]):
        start = 0 if colors[i] == "Black" else 1
        player_diffs = diffs.iloc[i, :].dropna()[start::2].values
        if colors[i] == "White":
            player_diffs *= -1
        player_diffs = player_diffs.clip(min=0)
        all_diffs.append(player_diffs)
    df = pd.DataFrame(all_diffs)
    df.to_csv(f"output/cp_loss/{player}.csv")

In [4]:
pgns = Path("pgns")
#players = [pgn.stem for pgn in pgns.glob("*.pgn")]
players = ['carlsen', 'erigaisi', 'gukesh', 'nepo', 'niemann']
players

['carlsen', 'erigaisi', 'gukesh', 'nepo', 'niemann']

Calculate move-by-move centipawn loss.

In [ ]:
for player in players:
    print(player)
    get_cp_loss(player)

In [9]:
for player in players:
    loss = pd.read_csv(f"output/cp_loss/{player}.csv", index_col=0)
    game_losses = loss.mean(axis=1)
    df = pd.DataFrame(game_losses)
    df.to_csv(f"output/mean_cp_loss/{player}.csv")

In [41]:
for player in players:
    loss = pd.read_csv(f"output/cp_loss/{player}.csv", index_col=0)
    game_losses = loss.std(axis=1)
    df = pd.DataFrame(game_losses)
    df.to_csv(f"output/std_cp_loss/{player}.csv")

In [40]:
for player in players:
    md = pd.read_csv(f"output/metadata/{player}.csv", index_col=0)
    md['WElo'] = 0
    md['BElo'] = 0
    
    if player == 'carlsen':
        md['WElo'] = md[md['White']=='Carlsen,M']['WhiteElo']
        md['BElo'] = md[md['Black']=='Carlsen,M']['BlackElo']
    elif player == 'erigaisi':
        md['WElo'] = md[md['White']=='Erigaisi, Arjun']['WhiteElo']
        md['BElo'] = md[md['Black']=='Erigaisi, Arjun']['BlackElo']
    elif player == 'gukesh':
        md['WElo'] = md[md['White']=='Gukesh, D.']['WhiteElo']
        md['BElo'] = md[md['Black']=='Gukesh, D.']['BlackElo']
    elif player == 'nepo':
        md['WElo'] = md[md['White']=='Nepomniachtchi, I.']['WhiteElo']
        md['BElo'] = md[md['Black']=='Nepomniachtchi, I.']['BlackElo']
    elif player == 'niemann':
        md['WElo'] = md[md['White']=='Niemann, Hans Moke']['WhiteElo']
        md['BElo'] = md[md['Black']=='Niemann, Hans Moke']['BlackElo']
        
    df = md[['WElo', 'BElo']]
    df['Elo'] = df[['WElo', 'BElo']].sum(axis=1)
    df.to_csv(f"output/elo/{player}.csv")

C:\Users\Eric\AppData\Local\Temp\ipykernel_56572\428151413.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Elo'] = df[['WElo', 'BElo']].sum(axis=1)
C:\Users\Eric\AppData\Local\Temp\ipykernel_56572\428151413.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Elo'] = df[['WElo', 'BElo']].sum(axis=1)
C:\Users\Eric\AppData\Local\Temp\ipykernel_56572\428151413.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 